In [1]:
from pathlib import Path
import json
from functools import reduce
import math
import datetime as dt
import pytz 
from itertools import product
from collections import OrderedDict
import time
import sys

import requests
import numpy as np
import pandas as pd
import geopandas as gpd
import shapely.ops as so
from requests_html import HTMLSession
    
import helpers as hp

%load_ext autoreload
%autoreload 2


In [8]:
def get_rent_data_urls():
    """
    Scrape MBIE's website to get the location of three files of rental bond data.
    Return a list of three dictionaries, each with the following keys and values.
    
    - ``'kind'``: the kind of data; one of 'rent_count', 'rent_mean', 'rent_geo_mean'
    - ``'filename'``: the name of the CSV data file
    - ``'url'``: the URL of the CSV data file
    
    """
    src_url = "https://www.mbie.govt.nz/building-and-energy/tenancy-and-housing/rental-bond-data/"

    # Scrape the HTML page above to get the data urls of interest
    session = HTMLSession()
    r = session.get(src_url)
    urls = [
        link for link in r.html.absolute_links 
        if "Quarterly" in link 
        and link.split("/")[-1].startswith("detailed-")
    ]
    
    # Filter urls to finer set and include some metadata
    return [
        {
            "kind": "rent_count",
            "filename": "detailed-lodged-bonds.csv",
            "url": [u for u in urls if u.endswith("detailed-lodged-bonds.csv")][0],
            "target_path": hp.DATA_DIR/'collected'/"detailed-lodged-bonds.csv",
        },
        {
            "kind": "rent_mean",
            "filename": "detailed-mean-rents.csv",
            "url": [u for u in urls if u.endswith("detailed-mean-rents.csv")][0],
            "target_path": hp.DATA_DIR/'collected'/"detailed-mean-rents.csv",
        },
        {
            "kind": "rent_geo_mean",
            "filename": "detailed-geo-mean-rents.csv",
            "url": [u for u in urls if u.endswith("detailed-geo-mean-rents.csv")][0],
            "target_path": hp.DATA_DIR/'collected'/"detailed-geo-mean-rents.csv",
        },
    ]


# Download rent data

In [10]:
data = get_rent_data_urls()
print(data)
    
for d in data:
    r = requests.get(d['url'])
    if not r.ok:
        print("Failed to get", d['filename'])
    else:
        print("Getting and saving", d['filename'])
        path = d["target_path"]
        with path.open('w') as tgt:
            tgt.write(r.text)


Getting and saving detailed-lodged-bonds.csv
Getting and saving detailed-mean-rents.csv
Getting and saving detailed-geo-mean-rents.csv


# Process rent data

In [12]:
# Reshape and merge all rent data sets

def clean(f, kind):
    f = f.copy()
    f = f.rename(columns={
        'SAU': 'au2001',
        'Property_Type': 'property_type',
        'Bedrooms': '#bedrooms'
    })

    # Drop subtotals
    cond = False
    for col in ['au2001', 'property_type', '#bedrooms']:
        cond |= f[col].str.contains('total', case=False)

    f = f[~cond].copy()
    
    # Reshape
    id_vars = ['au2001', 'property_type', '#bedrooms']
    value_vars = [c for c in f.columns if '-' in c]
    f = pd.melt(f, id_vars=id_vars, value_vars=value_vars,
      var_name='quarter', value_name=kind)
    
    return f

frames = []
for d in data:
    path = d["target_path"]
    print(path)
    f = pd.read_csv(path, dtype={'SAU': str})
    frames.append(clean(f, d["kind"]))
    
f = reduce(lambda x, y: pd.merge(x, y), frames)

# Merge in region data
path = hp.get_path('au2001_csv')
g = pd.read_csv(path, dtype={'au2001': str})
f = f.merge(g)

# Write to file
path = hp.get_path('rents')
f.to_csv(path, index=False)
f[f['rent_count'].notnull()].tail()


/home/araichev/affordability_nz/data/collected/detailed-lodged-bonds.csv
/home/araichev/affordability_nz/data/collected/detailed-mean-rents.csv
/home/araichev/affordability_nz/data/collected/detailed-geo-mean-rents.csv


,au2001,property_type,#bedrooms,quarter,rent_count,rent_mean,rent_geo_mean,au_name,territory,region,rental_area
1026382,612802,House,3,2009-06-01,5.0,188.0,184.0,Riverton West,Southland District,Southland,Clutha/Southland
1026406,612802,House,3,2011-06-01,6.0,198.0,195.0,Riverton West,Southland District,Southland,Clutha/Southland
1026427,612802,House,3,2013-03-01,6.0,218.0,213.0,Riverton West,Southland District,Southland,Clutha/Southland
1026433,612802,House,3,2013-09-01,6.0,207.0,205.0,Riverton West,Southland District,Southland,Clutha/Southland
1026457,612802,House,3,2015-09-01,5.0,230.0,228.0,Riverton West,Southland District,Southland,Clutha/Southland


In [13]:
# Print latest 2 quarters
hp.get_latest_quarters(2)

['2019-03-01', '2019-06-01']

# Explore rents

In [14]:
rents = hp.get_data('rents')
print(rents['quarter'].unique())
rents.head()


['1993-03-01' '1993-06-01' '1993-09-01' '1993-12-01' '1994-03-01'
 '1994-06-01' '1994-09-01' '1994-12-01' '1995-03-01' '1995-06-01'
 '1995-09-01' '1995-12-01' '1996-03-01' '1996-06-01' '1996-09-01'
 '1996-12-01' '1997-03-01' '1997-06-01' '1997-09-01' '1997-12-01'
 '1998-03-01' '1998-06-01' '1998-09-01' '1998-12-01' '1999-03-01'
 '1999-06-01' '1999-09-01' '1999-12-01' '2000-03-01' '2000-06-01'
 '2000-09-01' '2000-12-01' '2001-03-01' '2001-06-01' '2001-09-01'
 '2001-12-01' '2002-03-01' '2002-06-01' '2002-09-01' '2002-12-01'
 '2003-03-01' '2003-06-01' '2003-09-01' '2003-12-01' '2004-03-01'
 '2004-06-01' '2004-09-01' '2004-12-01' '2005-03-01' '2005-06-01'
 '2005-09-01' '2005-12-01' '2006-03-01' '2006-06-01' '2006-09-01'
 '2006-12-01' '2007-03-01' '2007-06-01' '2007-09-01' '2007-12-01'
 '2008-03-01' '2008-06-01' '2008-09-01' '2008-12-01' '2009-03-01'
 '2009-06-01' '2009-09-01' '2009-12-01' '2010-03-01' '2010-06-01'
 '2010-09-01' '2010-12-01' '2011-03-01' '2011-06-01' '2011-09-01'
 '2011-12-

,au2001,property_type,#bedrooms,quarter,rent_count,rent_mean,rent_geo_mean,au_name,territory,region,rental_area
0,500100,House,2,1993-03-01,NaN,NaN,NaN,Awanui,Far North District,Northland,Rural Far North
1,500100,House,3,1993-03-01,NaN,NaN,NaN,Awanui,Far North District,Northland,Rural Far North
2,500100,House,2,1993-06-01,NaN,NaN,NaN,Awanui,Far North District,Northland,Rural Far North
3,500100,House,3,1993-06-01,NaN,NaN,NaN,Awanui,Far North District,Northland,Rural Far North
4,500100,House,2,1993-09-01,NaN,NaN,NaN,Awanui,Far North District,Northland,Rural Far North


In [15]:
# Slice in time and aggregate 
agg_rents = hp.aggregate_rents(rents, '2018-06-01')
agg_rents.head()

/home/araichev/affordability_nz/py/helpers.py:200: FutureWarning: Interpreting tuple 'by' as a list of keys, rather than a single key. Use 'by=[...]' instead of 'by=(...)'. In the future, a tuple will always mean a single key.
  g = f.groupby(groupby_cols).apply(my_agg).reset_index()
/home/araichev/affordability_nz/py/helpers.py:190: RuntimeWarning: invalid value encountered in double_scalars
  d['rent_count']


,rental_area,#bedrooms,territory,region,rent_count,rent_mean,rent_geo_mean
0,Addington,1,Christchurch City,Canterbury,302.0,212.238411,206.186202
1,Addington,2,Christchurch City,Canterbury,164.0,331.664634,324.393268
2,Addington,3,Christchurch City,Canterbury,109.0,429.660550,423.572382
3,Addington,4,Christchurch City,Canterbury,11.0,441.545455,424.220800
4,Addington,5+,Christchurch City,Canterbury,0.0,NaN,NaN


In [16]:
f = hp.aggregate_rents(rents, '2018-06-01', groupby_cols=('au2001', '#bedrooms'))
cond = f['region'] == 'Auckland'
f[cond]

,au2001,#bedrooms,territory,region,rent_count,rent_mean,rent_geo_mean
289,505300,1,Rodney District,Auckland,0.0,NaN,NaN
290,505300,2,Rodney District,Auckland,10.0,395.000000,391.347672
291,505300,3,Rodney District,Auckland,42.0,446.809524,444.350686
292,505300,4,Rodney District,Auckland,12.0,462.500000,461.228455
293,505300,5+,Rodney District,Auckland,0.0,NaN,NaN
294,505400,1,Rodney District,Auckland,0.0,NaN,NaN
295,505400,2,Rodney District,Auckland,0.0,NaN,NaN
296,505400,3,Rodney District,Auckland,0.0,NaN,NaN
297,505400,4,Rodney District,Auckland,0.0,NaN,NaN
298,505500,1,Rodney District,Auckland,12.0,333.000000,321.775946


In [17]:
# What fraction of rental data do we have by #bedrooms?

def hits(group):
    d = {}
    d['hit_frac'] = group['rent_mean'].count()/group['rent_mean'].shape[0]
    return pd.Series(d)

date = '2018-06-01'
f = hp.aggregate_rents(rents, date, groupby_cols=('au2001', '#bedrooms'))
cond = f['region'] == 'Auckland'
print('census area units')
print(f[cond].copy().groupby('#bedrooms').apply(hits).reset_index())

f = hp.aggregate_rents(rents, date, groupby_cols=('rental_area', '#bedrooms'))
cond = f['region'] == 'Auckland'
print('rental area units')
print(f[cond].copy().groupby('#bedrooms').apply(hits).reset_index())


census area units
  #bedrooms  hit_frac
0         1  0.408669
1         2  0.713873
2         3  0.886040
3         4  0.639769
4        5+  0.157729
rental area units
  #bedrooms  hit_frac
0         1  0.787879
1         2  0.979798
2         3  1.000000
3         4  0.909091
4        5+  0.353535


# Select latest two quarters and slice into regional chunks

In [18]:
# Get latest two quarters of rents
start_date = hp.get_latest_quarters(2)[0]
rents = hp.get_data('rents')
rents = rents[rents['quarter'] >= start_date].copy()

# Create regional slices
for region in hp.REGIONS:
    # Build rents
    region_c = region.capitalize()
    region_rents = rents[rents['region'] == region_c].copy()
    print(region, region_rents['quarter'].unique(), '#rows =', region_rents.shape[0])
    path = hp.get_path('rents', region)
    print('  Saving to', path)
    
    # Create region directory if it does not exist
    if not path.parent.exists():
        path.parent.mkdir()
        
    region_rents.to_csv(path, index=False)
    

auckland ['2019-03-01' '2019-06-01'] #rows = 5620
  Saving to /home/araichev/affordability_nz/data/processed/auckland/rents.csv
canterbury ['2019-03-01' '2019-06-01'] #rows = 2560
  Saving to /home/araichev/affordability_nz/data/processed/canterbury/rents.csv
wellington ['2019-03-01' '2019-06-01'] #rows = 2428
  Saving to /home/araichev/affordability_nz/data/processed/wellington/rents.csv


# JSONize regional rents for the web, grouping by rental area and number of bedrooms

In [19]:
for region in hp.REGIONS:
    region_rents = hp.get_data('rents', region)
    d = hp.build_json_rents(region_rents)
    path = hp.get_path('rents_json', region)
    print('**', region, '\n', d)
    with path.open('w') as tgt:
        json.dump(d, tgt)

    

** auckland 
 {'Albany': {'1': 282.0, '2': 516.0, '3': 597.0, '4': 681.0}, 'Avondale': {'1': 193.0, '2': 462.0, '3': 570.0, '4': 669.0}, 'Balmoral': {'1': 358.0, '2': 511.0, '3': 711.0, '4': 901.0}, 'Beachhaven/Birkdale': {'1': 362.0, '2': 449.0, '3': 569.0, '4': 701.0}, 'Blockhouse Bay/New Windsor': {'1': None, '2': 490.0, '3': 569.0, '4': 691.0}, 'Botony Downs': {'1': None, '2': None, '3': 619.0, '4': 723.0}, 'Browns Bay': {'1': 487.0, '2': 477.0, '3': 614.0, '4': 774.0}, 'Bucklands Beach': {'1': None, '2': 458.0, '3': 617.0, '4': 727.0}, 'Central East': {'1': 392.0, '2': 549.0, '3': 686.0, '4': None}, 'Central West': {'1': 403.0, '2': 511.0, '3': 663.0, '4': None}, 'Chatswood/Birkenhead/Northcote Point': {'1': 413.0, '2': 522.0, '3': 652.0, '4': 873.0}, 'Dannemora': {'1': 419.0, '2': 513.0, '3': 634.0, '4': 702.0}, 'Devonport': {'1': 437.0, '2': 585.0, '3': 754.0, '4': None}, 'East Coast Bays': {'1': 361.0, '2': 576.0, '3': 686.0, '4': 966.0}, 'Eden Terrace': {'1': 417.0, '2': 568.0

** wellington 
 {'Brooklyn': {'1': 398.0, '2': 512.0, '3': 685.0, '4': 743.0}, 'Carterton/South Wairarapa': {'1': None, '2': None, '3': 395.0, '4': None}, 'Eastern Bays': {'1': None, '2': 424.0, '3': 625.0, '4': None}, 'Epuni/Avalon': {'1': 179.0, '2': 443.0, '3': 554.0, '4': None}, 'Hataitai': {'1': 358.0, '2': 505.0, '3': 718.0, '4': 805.0}, 'Heretaunga/Silverstream': {'1': None, '2': 386.0, '3': 543.0, '4': 647.0}, 'Hutt Central/Waterloo': {'1': 289.0, '2': 488.0, '3': 593.0, '4': None}, 'Island Bay/Melrose': {'1': 265.0, '2': 481.0, '3': 651.0, '4': 857.0}, 'Johnsonville/Newlands': {'1': 365.0, '2': 443.0, '3': 580.0, '4': 681.0}, 'Karori': {'1': 382.0, '2': 517.0, '3': 629.0, '4': 781.0}, 'Karori South/Makara': {'1': 235.0, '2': 391.0, '3': 640.0, '4': 696.0}, 'Kelburn/Aro Valley': {'1': 357.0, '2': 521.0, '3': 678.0, '4': 854.0}, 'Khandallah': {'1': 373.0, '2': None, '3': 645.0, '4': 878.0}, 'Kilbirnie/Lyall Bay': {'1': 383.0, '2': 487.0, '3': 666.0, '4': 705.0}, 'Kingston/Happy 